In [ ]:
# Author: Ying Yang

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
raw_data = pd.read_csv("Inventory_sample.csv")
raw_data.head()

In [ ]:
slot_raw_data = pd.read_csv("slot_sample.csv")
slot_raw_data.head()

## Data Exploration

In [ ]:
raw_data.info()

In [ ]:
print(raw_data.shape)
raw_data.dtypes

In [ ]:
# get the number of missing data points per column
raw_data.isnull().sum()

In [ ]:
# look at the values for the categories
print("unique values for Major Super Category :\n", raw_data["Major Category"].unique(),"\n")	

In [ ]:
# look at the total qty in slot for each item.
item_ttl_qty = raw_data.groupby("Item ID")['Qty in Slot'].sum()
item_ttl_qty

## Cube Utilization

In [ ]:
# ------------------ Data Tranformation ------------------
# only select items that are stored at WH01
raw_data_wh01 = raw_data.loc[raw_data["WH"]==1]

# join the grocery sheet with FR03 by location 
raw_data_wh01 = raw_data_wh01.merge(slot_raw_data_wh01, how="left",left_on='Location (Slot Address)', right_on='LOCATION')

# only select the columns we need
raw_data_wh01 = raw_data_wh01[['DC_x', 'WH', ... 'Date']]


In [ ]:
# ------------------ CALCULATING NUMBER OF LAYERS ------------------
# calculate the supposed layer for each product based on its quantity in slot and DC TI
raw_data_wh01['Cal_Number_of_Layer'] = raw_data_wh01['Qty in Slot'] / raw_data_wh01['DC Ti']

# round the value up
raw_data_wh01['Cal_Number_of_Layer'] = raw_data_wh01['Cal_Number_of_Layer'].apply(np.ceil)

In [ ]:
# ------------------ CALCULATING NUMBER OF POSITIONS (DOUBLE/TRIPLE...) ------------------
# for most of data, either pick location has value or reserve location has value, thus we could simply add them up
raw_data_wh01["Number_of_Positions"] = raw_data_wh01["pick_location"] + raw_data_wh01["reserve_location"]

# check abnormal values
raw_data_wh01["Number_of_Positions"].unique()


In [ ]:
# look at the records that have 0 position
raw_data_wh01.loc[raw_data_wh01["Number_of_Positions"]==0]

In [ ]:
# exclude the abnormal data
raw_data_wh01 = raw_data_wh01.loc[raw_data_wh01['Number_of_Positions']!=0]

In [ ]:
# raw_data_wh01.loc[raw_data_wh01["Loc Height"] != raw_data_wh01["SelHGT"]+raw_data_wh01["ResHGT"]]

In [ ]:
# ------------------ CALCULATING %CUBE UTILIZATION ------------------
raw_data_wh01["Product_Width_Assumed"] = product_width
raw_data_wh01["Product_Length_Assumed"] = product_length
raw_data_wh01["Pallet_Width_Assumed"] = pallet_width
raw_data_wh01["Pallet_Length_Assumed"] = pallet_length

# calculate cube in feet
raw_data_wh01["Product_Cube_in_Feet"] = raw_data_wh01["Prod Height"]*raw_data_wh01["Product_Width_Assumed"]*raw_data_wh01["Product_Length_Assumed"]*raw_data_wh01['Cal_Number_of_Layer']/12**3

raw_data_wh01["Pallet_Cube_in_Feet"] = raw_data_wh01["Pallet Height"]*raw_data_wh01["Pallet_Width_Assumed"]*raw_data_wh01["Pallet_Length_Assumed"]/12**3

raw_data_wh01["Location_Cube_in_Feet"] = raw_data_wh01["Loc Height"]*raw_data_wh01["WDTH"]*raw_data_wh01["DPTH"]*raw_data_wh01["ResPOS"]/12**3

# if there is no product in this slot, then the pallet cube should also be 0
raw_data_wh01.loc[raw_data_wh01.Product_Cube_in_Feet==0,'Pallet_Cube_in_Feet']=0


# calculate cube utilization
raw_data_wh01["% Cube Utl"]=(raw_data_wh01["Product_Cube_in_Feet"]+raw_data_wh01["Pallet_Cube_in_Feet"])/raw_data_wh01["Location_Cube_in_Feet"]*100
raw_data_wh01["% Cube Utl"] = np.round(raw_data_wh01["% Cube Utl"],decimals=2)


In [ ]:
# rearrange the the order of columns
raw_data_wh01 = raw_data_wh01[['DC_x', 'WH', ...'Date']]

In [ ]:
raw_data_wh01.loc[raw_data_wh01.Slot_Type=='pick','% Cube Utl']=np.nan

In [ ]:
# only focus on locations whose heights are greater than 2
raw_data_wh01_loc_height_gt_two = raw_data_wh01.loc[raw_data_wh01["Loc Height"]> 2]

print("The percentage of DC01 WH01 items stored at locations whose heighsts are greater than 2 inchs are : ", round(raw_data_wh01_loc_height_gt_two["% Cube Utl"].count()/raw_data_wh01["% Cube Utl"].count()*100, 2),"%\n")

print("Descriptive statistics:", round(raw_data_wh01_loc_height_gt_two["% Cube Utl"].describe(),2))

In [ ]:
# ------------------ CALCULATING THE PERCENTAGE OF HOW MANY SLOTS THAT HAVE MORE THAN 80% CUBE UTILIZATION ------------------
cube_utl_gt_80_wh01_loc_height_gt_two_count = raw_data_wh01_loc_height_gt_two[raw_data_wh01_loc_height_gt_two["% Cube Utl"]>=80]["% Cube Utl"].count()
cube_utl_wh01_loc_height_gt_two_count = raw_data_wh01_loc_height_gt_two["% Cube Utl"].count()

print("The percentage of cube that has 80% or above utilization (only for items whose location height greater than 2):", round(cube_utl_gt_80_wh01_loc_height_gt_two_count/cube_utl_wh01_loc_height_gt_two_count*100,2),"%")

In [ ]:
# ------------------ CALCULATING THE PERCENTAGE OF HOW MANY SLOTS THAT HAVE MORE THAN 100% CUBE UTILIZATION ------------------
cube_utl_gt_100_wh01_loc_height_gt_two_count = raw_data_wh01_loc_height_gt_two[raw_data_wh01_loc_height_gt_two["% Cube Utl"]>100]["% Cube Utl"].count()

print("The percentage of cube that has more than 100% utilization (only for items whose location height greater than 2):", round(cube_utl_gt_100_wh01_loc_height_gt_two_count/cube_utl_wh01_loc_height_gt_two_count*100,2),"%")

In [ ]:
# ------------------ DEAL WITH OUTLIERS ------------------
# extract the data that have more than 100% cube utilization rate 
outlier_more_than_100 = raw_data_wh01_loc_height_gt_two[raw_data_wh01_loc_height_gt_two["% Cube Utl"]>100]
# outlier_more_than_100


outlier_more_than_100_qty_gt_full_pallet = outlier_more_than_100.loc[outlier_more_than_100["Qty in Slot"] > outlier_more_than_100["Full DC Pallet Qty"]]
print("Among the cases that have a more than 100% cube utilization, the number of cases whose qty in slot is bigger than Full DC Pallet Qty:", outlier_more_than_100_qty_gt_full_pallet.shape[0])
print("Among the cases that have a more than 100% cube utilization, the percentage of DC01 WH01 items stored at locations whose heighsts are greater than 2 inchs and whose qty in slot is bigger than Full DC Pallet Qty (in terms of all cases in DC01 WH01): ", round(outlier_more_than_100_qty_gt_full_pallet["% Cube Utl"].count()/raw_data_wh01["% Cube Utl"].count()*100, 2),"%\n")



outlier_qty_gt_full_pallet = raw_data_wh01_loc_height_gt_two.loc[raw_data_wh01_loc_height_gt_two["Qty in Slot"] > raw_data_wh01_loc_height_gt_two["Full DC Pallet Qty"]]
print("The number of cases whose qty in slot is bigger than Full DC Pallet Qty:", outlier_qty_gt_full_pallet.shape[0])
print("The percentage of DC01 WH01 items stored at locations whose heighsts are greater than 2 inchs and whose qty in slot is bigger than Full DC Pallet Qty (in terms of all cases in DC01 WH01):", round(outlier_qty_gt_full_pallet["% Cube Utl"].count()/raw_data_wh01["% Cube Utl"].count()*100, 2),"%\n")

In [ ]:
print(outlier_more_than_100_qty_gt_full_pallet['Slot Type'].value_counts())

In [ ]:
# QUESTIONS -- not quite sure about how to display this result, perhaps we can export this transformed data file and use power BI, this chart below is just for quick check
# ------------------ VISUALIZE THE OUTPUT ------------------
fig = plt.figure()
# Set title
plt.title("% Cube Utl Hist for DC01 WH01 Items Stored at Locations Whose Height >2")
  
# adding labels
plt.xlabel('% Cube Utl')
plt.ylabel('Number of items')
plt.hist(raw_data_wh01_loc_height_gt_two["% Cube Utl"])
plt.show

In [ ]:
# get the data part that %cube is smaller than 100
raw_data_wh01_loc_height_gt_two_utl_ls_100 = raw_data_wh01_loc_height_gt_two[raw_data_wh01_loc_height_gt_two["% Cube Utl"]<= 100]
# raw_data_wh01_loc_height_gt_two_utl_ls_100

# # get the data part that %cube is more than 100 due to some calculation problem and change it back to 100
# outlier_more_than_100_qty_eqls_full_pallet = outlier_more_than_100.loc[outlier_more_than_100["Qty in Slot"] <= outlier_more_than_100["Full DC Pallet Qty"]]
# outlier_more_than_100_qty_eqls_full_pallet["% Cube Utl"] = 100

# # combine these two datasets
# raw_data_wh01_loc_height_gt_two_combined = pd.concat([raw_data_wh01_loc_height_gt_two_utl_ls_100, outlier_more_than_100_qty_eqls_full_pallet])

In [ ]:
# QUESTIONS -- not quite sure about how to display this result, perhaps we can export this transformed data file and use power BI, this chart below is just for quick check
# ------------------ VISUALIZE THE OUTPUT ------------------
fig = plt.figure()
# Set title
plt.title("% Cube Utl Hist for DC01 WH01 Items Stored at Locations Whose Height >2 and utilization <= 100%")
  
# adding labels
plt.xlabel('% Cube Utl')
plt.ylabel('Number of items')
plt.hist(raw_data_wh01_loc_height_gt_two_utl_ls_100["% Cube Utl"])
plt.show

## Ti Hi mismatch

In [ ]:
tihi_df = raw_data_wh01[["Item_ID","DC Ti","DC Hi","Vendor Ti","Vendor Hi"]]

# Ti doesnt't fit
ti_mismatch = tihi_df.loc[tihi_df["DC Ti"] != tihi_df["Vendor Ti"] ]
# Hi doesnt't fit
hi_mismatch = tihi_df.loc[tihi_df["DC Hi"] != tihi_df["Vendor Hi"]]
# either Ti or Hi doesn't fit
ti_or_hi_mismatch = tihi_df.loc[(tihi_df["DC Ti"] != tihi_df["Vendor Ti"])|(tihi_df["DC Hi"] != tihi_df["Vendor Hi"])]

print("Ti mismatch rate:", round(ti_mismatch.shape[0]/tihi_df.shape[0]*100,2),"%\n")
print("Hi mismatch rate:", round(hi_mismatch.shape[0]/tihi_df.shape[0]*100,2),"%\n")
print("Ti or Hi mismatch rate:", round(ti_or_hi_mismatch.shape[0]/tihi_df.shape[0]*100,2),"%\n")

In [ ]:
raw_data_wh01['Is_Ti_Mismatch'] = (raw_data_wh01["DC Ti"] != raw_data_wh01["Vendor Ti"]).astype(int)
raw_data_wh01['Is_Hi_Mismatch'] = (raw_data_wh01["DC Hi"] != raw_data_wh01["Vendor Hi"]).astype(int)
raw_data_wh01['Is_Ti_or_Hi_Mismatch'] = ((raw_data_wh01["DC Ti"] != raw_data_wh01["Vendor Ti"]) | (raw_data_wh01["DC Ti"] != raw_data_wh01["Vendor Ti"])).astype(int)
raw_data_wh01

## Slot Occupancy

In [ ]:
slot_occupied_num = raw_data_wh01.loc[raw_data_wh01["Qty"]!=0].shape[0]
print("The number of occupied slots : ", slot_occupied_num)

total_slot_num = len(slot_raw_data_wh01["LOCATION"].unique())
print("The number of total slots : ", total_slot_num)

slot_occupied_per = slot_occupied_num/total_slot_num * 100

print("The percentage of slot occupied is {:.2f}".format(slot_occupied_per), "%")

In [ ]:
print("the number of occupied pick locations is ", raw_data_wh01.loc[(raw_data_wh01["Qty"]!=0) & (raw_data_wh01["Slot Type"] == "pick")].shape[0])
print("the number of all pick locations is ", raw_data_wh01.loc[raw_data["Slot Type"] == "pick"].shape[0])

In [ ]:
reserve_occupied_num = raw_data_wh01.loc[raw_data_wh01["Slot Type"] == "reserve"].shape[0]
reserve_occupied_per = reserve_occupied_num/total_slot_num * 100

print("The percentage of reserve slot occupied is {:.2f}".format(reserve_occupied_per), "%")

# the unit is the number of reserve slots
# I have examined the data, there is no free reserve location

In [ ]:
ttl_qty_slot = raw_data_wh01['Qty'].sum()
avg_case_per_slot = ttl_qty_slot/total_slot_num
avg_case_per_occupied_slot = ttl_qty_slot/slot_occupied_num
print("Average cases per slot(total slot number) is {:.2f}".format(avg_case_per_slot))
print("Average cases per slot(only occupied slot number) is {:.2f}".format(avg_case_per_occupied_slot))

In [ ]:
ttl_qty_select_slot = raw_data_wh01.loc[raw_data_wh01["Slot Type"] == "pick"]['Qty'].sum()

select_slot_num = slot_raw_data_wh01.loc[slot_raw_data_wh01["Slot Type"] == "pick"]["pick_position"].sum()
select_occupied_slot_num = raw_data_wh01.loc[(raw_data_wh01["Qty"]!=0) & (raw_data_wh01["Slot Type"] == "pick")].shape[0]

avg_case_per_select_slot = ttl_qty_select_slot/select_slot_num
avg_case_per_occupied_select_slot = ttl_qty_select_slot/select_occupied_slot_num

print("Average cases per select slot(total slot number) is {:.2f}".format(avg_case_per_select_slot))
print("Average cases per select (only occupied slot number) is {:.2f}".format(avg_case_per_occupied_select_slot))

In [ ]:
ttl_qty_reserve_slot = raw_data_wh01.loc[raw_data["Slot Type"] == "reserve"]['Qty'].sum()

reserve_slot_num = slot_raw_data_wh01.loc[slot_raw_data_wh01["Slot Type"] == "pick"]["reserve_position"].sum()
reserve_occupied_slot_num = raw_data_wh01.loc[(raw_data_wh01["Qty"]!=0) & (raw_data_wh01["Slot Type"] == "reserve")].shape[0]

avg_case_per_reserve_slot = ttl_qty_reserve_slot/reserve_slot_num
avg_case_per_occupied_reserve_slot = ttl_qty_reserve_slot/reserve_occupied_slot_num

print("Average cases per reserve slot(total slot number) is {:.2f}".format(avg_case_per_reserve_slot))
print("Average cases per reserve (only occupied slot number) is {:.2f}".format(avg_case_per_occupied_reserve_slot))

In [ ]:
ttl_qty_slot_classA = raw_data_wh01.loc[raw_data_wh01["CYCLE CLASS"] == "A"]['Qty'].sum()
total_slot_num_classA = len(raw_data_wh01.loc[raw_data_wh01["CYCLE CLASS"] == "A"]["Location (Slot Address)"].unique())
slot_occupied_num_classA = raw_data_wh01.loc[(raw_data_wh01["Qty"]!=0) & (raw_data_wh01["CYCLE CLASS"] == "A")].shape[0]

avg_case_per_slot_classA = ttl_qty_slot_classA/total_slot_num_classA
avg_case_per_occupied_slot_classA = ttl_qty_slot_classA/slot_occupied_num_classA
print("Average cases per slot(total slot number) for A movement products is {:.2f}".format(avg_case_per_slot_classA))
print("Average cases per slot(only occupied slot number) for A movement products is {:.2f}".format(avg_case_per_occupied_slot_classA))

# Height Group

In [ ]:
# 10 slots' height in inventory don't fit with those in slot file
loc_height_mismatch = raw_data_wh01.loc[raw_data_wh01["Loc Height"] != raw_data_wh01["pick_height"]+raw_data_wh01["reserve_height"]]

In [ ]:
slot_raw_data_wh01_select = slot_raw_data_wh01.loc[slot_raw_data_wh01["slot_type"]=="pick"]

select_loc_hight_df = pd.DataFrame(columns=['Height','Count of height'])

select_loc_hight_list = slot_raw_data_wh01_select["pick_height"].unique()

for height in select_loc_hight_list:
  count = slot_raw_data_wh01_select.loc[slot_raw_data_wh01_select["pick_height"]==height]["pick_location"].sum()
  new_row = {'Height':height, 'Count of height':count}
  select_loc_hight_df = select_loc_hight_df.append(new_row, ignore_index=True)

select_loc_hight_df = select_loc_hight_df.sort_values(by=["Count of height"],ascending=0)
select_loc_hight_df["%"] = select_loc_hight_df["Count of height"]/select_loc_hight_df["Count of height"].sum()*100
select_loc_hight_df

In [ ]:
fig = plt.figure()
# Set title
plt.title("Hist of Select Location Heights")
  
# adding labels
plt.xlabel('Select Location Height')
plt.ylabel('Frequency')
plt.bar(select_loc_hight_df["Height"],select_loc_hight_df["Count of height"])
plt.show

In [ ]:
slot_raw_data_wh01_reserve = slot_raw_data_wh01.loc[slot_raw_data_wh01["slot_type"]=="reserve"]

reserve_loc_hight_df = pd.DataFrame(columns=['Height','Count of height'])

reserve_loc_hight_list = slot_raw_data_wh01_reserve["reserve_hight"].unique()

for height in reserve_loc_hight_list:
  count = slot_raw_data_wh01_reserve.loc[slot_raw_data_wh01_reserve["reserve_hight"]==height]["reserve_position"].sum()
  new_row = {'Height':height, 'Count of height':count}
  reserve_loc_hight_df = reserve_loc_hight_df.append(new_row, ignore_index=True)

reserve_loc_hight_df = reserve_loc_hight_df.sort_values(by=["Count of height"],ascending=0)
reserve_loc_hight_df["%"] = reserve_loc_hight_df["Count of height"]/reserve_loc_hight_df["Count of height"].sum()*100
reserve_loc_hight_df

In [ ]:
reserve_loc_hight_df_ls_999 = reserve_loc_hight_df.loc[reserve_loc_hight_df["Height"]<999]

fig = plt.figure()
# Set title
plt.title("Hist of Reserve Location Heights")
  
# adding labels
plt.xlabel('Reserve Location Height')
plt.ylabel('Frequency')
plt.bar(reserve_loc_hight_df_ls_999["Height"],reserve_loc_hight_df_ls_999["Count of height"])
plt.show

In [ ]:
import scipy.stats as stats
from scipy.stats import mannwhitneyu

In [ ]:
X = raw_data_wh01.loc[(raw_data_wh01["Slot Type"]=="pick") & (raw_data_wh01["Loc Height"]<999),"Loc Height"].values
Y = raw_data_wh01.loc[(raw_data_wh01["Slot Type"]=="reserve") & (raw_data_wh01["Loc Height"]<999),"Loc Height"].values
print(stats.mannwhitneyu(x=X, y=Y, alternative = 'two-sided'))
print(stats.mannwhitneyu(x=X, y=Y, alternative = 'less'))
print(stats.mannwhitneyu(x=X, y=Y, alternative = 'greater'))

## OFF AISLE

In [ ]:
raw_data_wh01_pd = raw_data_wh01
raw_data_wh01_pd.rename(columns={'Item': 'Item_ID', 'Slot Type': 'Slot_Type'}, inplace=True)

In [ ]:
item_list = list(set(raw_data_wh01_pd["Item_ID"].unique()))
raw_data_wh01_pd["off_aisle"]=0

for item in item_list:
  pick_list = list(set(raw_data_wh01_pd.loc[(raw_data_wh01_pd["Item_ID"]==item) & (raw_data_wh01_pd["Slot_Type"]=="pick")]["Aisle"]))
  reserve_list = list(set(raw_data_wh01_pd.loc[(raw_data_wh01_pd["Item_ID"]==item) & (raw_data_wh01_pd["Slot_Type"]=="reserve")]["Aisle"]))
  for reserve_aisle in reserve_list:
    if reserve_aisle not in pick_list:
      raw_data_wh01_pd.loc[(raw_data_wh01_pd["Item_ID"]==item) & (raw_data_wh01_pd["Slot_Type"]=="reserve") & (raw_data_wh01_pd["Aisle"]==reserve_aisle), "off_aisle"] = 1

## Extra codes

In [ ]:
# def cal_cube_product_in_feet(product_height , product_width , product_length , num_of_layer):
#   product_in_inch = product_height * product_width * product_length * num_of_layer
#   product_in_feet = product_in_inch/12^3
#   return product_in_feet

# def cal_cube_pallet_in_feet(pallet_height , pallet_width , pallet_length):
#   pallet_in_inch = pallet_height * pallet_width * pallet_length
#   pallet_in_feet = pallet_in_inch/12^3
#   return pallet_in_feet

# def cal_cube_location_in_feet(location_height , location_width , location_length):
#   location_in_inch = location_height * location_width * location_length
#   location_in_feet = location_in_inch/12^3
#   return location_in_feet

# def cal_per_cube_utl():
#   product_cube = cal_cube_product_in_feet(product_height , product_width , product_length , num_of_layer)
#   pallet_cube = cal_cube_pallet_in_feet(pallet_height , pallet_width , pallet_length)
#   location_cube = cal_cube_location_in_feet(location_height , location_width , location_length)
#   per_cube_utl = ( product_cube + pallet_cube)/location_cube * 100
#   return per_cube_utl